In [11]:
# Import library dengan cara yang lebih spesifik untuk menghindari konflik
from pyspark.sql import SparkSession
from pyspark.sql import functions as F  # Menggunakan alias untuk menghindari konflik
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("🔧 Menggunakan import yang lebih spesifik untuk menghindari konflik nama")

# Inisialisasi Spark Session
spark = SparkSession.builder \
    .appName("Amazon Data Exploration Fixed") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

print("🚀 Spark Session berhasil dibuat!")
print(f"Spark Version: {spark.version}")

# Membaca data CSV dengan error handling yang lebih baik
try:
    df = spark.read.option("header", "true") \
        .option("inferSchema", "true") \
        .option("multiline", "true") \
        .option("escape", '"') \
        .option("quote", '"') \
        .csv("amazon - Copy.csv")

    print(f"✅ Data berhasil dimuat: {df.count()} baris")
except Exception as e:
    print(f"❌ Error saat membaca file: {e}")
    # Coba dengan opsi yang lebih sederhana
    df = spark.read.option("header", "true").csv("amazon.csv")
    print("✅ Data dimuat dengan opsi sederhana")

print("\n📊 INFORMASI DASAR DATASET")
print("=" * 50)
print(f"Jumlah Baris: {df.count()}")
print(f"Jumlah Kolom: {len(df.columns)}")

# Cek kolom yang tersedia
print("\n📋 KOLOM YANG TERSEDIA:")
for i, column in enumerate(df.columns, 1):
    print(f"{i:2d}. {column}")

# Menampilkan schema dengan penanganan yang lebih baik
print("\n🔍 SCHEMA DATASET")
print("=" * 50)
df.printSchema()

# Tampilkan sample data
print("\n📋 SAMPLE DATA (3 baris pertama)")
print("=" * 50)
df.show(3, truncate=True)

# PEMBERSIHAN DATA DENGAN METODE YANG LEBIH ROBUST
print("\n🧹 PROSES PEMBERSIHAN DATA")
print("=" * 50)

# Menggunakan F.col() instead of col() untuk menghindari konflik
try:
    # Membersihkan kolom harga dengan regex yang lebih spesifik
    df_clean = df.withColumn("discounted_price_clean",
                            F.regexp_replace(F.col("discounted_price"), "[₹,]", "")) \
                 .withColumn("actual_price_clean",
                            F.regexp_replace(F.col("actual_price"), "[₹,]", "")) \
                 .withColumn("discount_percentage_clean",
                            F.regexp_replace(F.col("discount_percentage"), "%", ""))

    print("✅ Pembersihan teks berhasil")

    # Konversi ke numeric dengan error handling
    df_clean = df_clean.withColumn("discounted_price_numeric",
                                  F.col("discounted_price_clean").cast("double")) \
                       .withColumn("actual_price_numeric",
                                  F.col("actual_price_clean").cast("double")) \
                       .withColumn("discount_percentage_numeric",
                                  F.col("discount_percentage_clean").cast("double")) \
                       .withColumn("rating_numeric",
                                  F.col("rating").cast("double"))

    print("✅ Konversi numeric berhasil")

except Exception as e:
    print(f"❌ Error dalam pembersihan data: {e}")
    print("🔄 Mencoba metode alternatif...")

    # Metode alternatif jika regex gagal
    df_clean = df.select("*")
    for column in ["discounted_price", "actual_price"]:
        if column in df.columns:
            df_clean = df_clean.withColumn(f"{column}_numeric",
                                         F.regexp_replace(F.col(column), "[^0-9.]", "").cast("double"))

# Cek hasil pembersihan data
print("\n🔍 CEK HASIL PEMBERSIHAN")
print("=" * 50)

# Tampilkan kolom yang berhasil dibuat
numeric_columns = [col for col in df_clean.columns if "numeric" in col]
print(f"Kolom numeric yang berhasil dibuat: {numeric_columns}")

# Statistik deskriptif untuk kolom yang berhasil
if numeric_columns:
    try:
        print("\n📊 STATISTIK DESKRIPTIF")
        df_clean.select(*numeric_columns).describe().show()
    except Exception as e:
        print(f"Error dalam statistik: {e}")

# ANALISIS YANG LEBIH ROBUST
print("\n🏷️ ANALISIS KATEGORI PRODUK")
print("=" * 50)

if "category" in df.columns:
    try:
        category_analysis = df_clean.groupBy("category") \
                                   .count() \
                                   .orderBy(F.desc("count"))

        print("Top 10 Kategori:")
        category_analysis.show(10, truncate=False)

        # Simpan untuk visualisasi
        category_pandas = category_analysis.limit(10).toPandas()

    except Exception as e:
        print(f"Error dalam analisis kategori: {e}")

# ANALISIS RATING
print("\n⭐ ANALISIS RATING")
print("=" * 50)

if "rating" in df.columns:
    try:
        rating_analysis = df_clean.filter(F.col("rating").isNotNull()) \
                                 .groupBy("rating") \
                                 .count() \
                                 .orderBy("rating")

        rating_analysis.show()

        # Konversi untuk visualisasi
        rating_pandas = rating_analysis.toPandas()

    except Exception as e:
        print(f"Error dalam analisis rating: {e}")

# EKSTRAKSI BRAND DENGAN METODE YANG LEBIH ROBUST
print("\n🏢 ANALISIS BRAND")
print("=" * 50)

if "product_name" in df.columns:
    try:
        # Daftar brand yang akan dicari
        brand_patterns = {
            "boAt": "boAt",
            "Amazon": "Amazon|AmazonBasics",
            "MI": "MI|Mi",
            "Samsung": "Samsung",
            "OnePlus": "OnePlus",
            "TP-Link": "TP-Link",
            "Portronics": "Portronics",
            "Ambrane": "Ambrane",
            "Duracell": "Duracell",
            "LG": "LG"
        }

        # Buat kolom brand dengan case-when yang lebih fleksibel
        brand_condition = F.when(F.col("product_name").rlike("(?i)boat"), "boAt")

        for brand, pattern in brand_patterns.items():
            if brand != "boAt":  # Skip boAt karena sudah ditangani
                brand_condition = brand_condition.when(
                    F.col("product_name").rlike(f"(?i){pattern}"), brand
                )

        brand_condition = brand_condition.otherwise("Others")

        df_with_brand = df_clean.withColumn("brand", brand_condition)

        brand_analysis = df_with_brand.groupBy("brand") \
                                     .count() \
                                     .orderBy(F.desc("count"))

        print("Distribusi Brand:")
        brand_analysis.show()

        # Simpan untuk visualisasi
        brand_pandas = brand_analysis.toPandas()

    except Exception as e:
        print(f"Error dalam analisis brand: {e}")

# KONVERSI KE PANDAS UNTUK VISUALISASI
print("\n📊 PERSIAPAN VISUALISASI")
print("=" * 50)

try:
    # Filter data yang valid untuk visualisasi
    valid_data = df_clean.filter(F.col("discounted_price_numeric").isNotNull() &
                                F.col("rating_numeric").isNotNull())

    # Konversi ke pandas dengan sampel jika data terlalu besar
    if valid_data.count() > 1000:
        sample_data = valid_data.sample(0.8, seed=42)  # Ambil 80% sample
        pdf = sample_data.toPandas()
        print(f"✅ Menggunakan sample data: {len(pdf)} records")
    else:
        pdf = valid_data.toPandas()
        print(f"✅ Menggunakan full data: {len(pdf)} records")

    # VISUALISASI DENGAN PLOTLY
    print("\n🎨 MEMBUAT VISUALISASI")
    print("=" * 50)

    # 1. Distribusi Harga (jika kolom ada)
    if 'discounted_price_numeric' in pdf.columns and not pdf['discounted_price_numeric'].isna().all():
        fig1 = px.histogram(pdf, x='discounted_price_numeric',
                           title='📊 Distribusi Harga Produk Amazon',
                           labels={'discounted_price_numeric': 'Harga (₹)', 'count': 'Jumlah Produk'},
                           nbins=20)
        fig1.update_layout(showlegend=False)
        fig1.show()
        print("✅ Histogram harga berhasil dibuat")

    # 2. Distribusi Rating
    if 'rating_numeric' in pdf.columns and not pdf['rating_numeric'].isna().all():
        fig2 = px.histogram(pdf, x='rating_numeric',
                           title='⭐ Distribusi Rating Produk',
                           labels={'rating_numeric': 'Rating', 'count': 'Jumlah Produk'},
                           nbins=10)
        fig2.show()
        print("✅ Histogram rating berhasil dibuat")

    # 3. Scatter plot Harga vs Rating
    if all(col in pdf.columns for col in ['discounted_price_numeric', 'rating_numeric']):
        # Filter outlier untuk visualisasi yang lebih baik
        pdf_filtered = pdf[(pdf['discounted_price_numeric'] < pdf['discounted_price_numeric'].quantile(0.95))]

        fig3 = px.scatter(pdf_filtered,
                         x='discounted_price_numeric',
                         y='rating_numeric',
                         title='💰 Hubungan Harga vs Rating',
                         labels={'discounted_price_numeric': 'Harga (₹)',
                                'rating_numeric': 'Rating'},
                         opacity=0.7)
        fig3.show()
        print("✅ Scatter plot berhasil dibuat")

except Exception as e:
    print(f"❌ Error dalam visualisasi: {e}")
    print("💡 Tip: Pastikan data memiliki nilai numeric yang valid")

# INSIGHT DAN REKOMENDASI
print("\n💡 RINGKASAN ANALISIS")
print("=" * 50)

try:
    total_products = df.count()
    valid_price_products = df_clean.filter(F.col("discounted_price_numeric").isNotNull()).count()
    valid_rating_products = df_clean.filter(F.col("rating_numeric").isNotNull()).count()

    print(f"📊 Total produk: {total_products}")
    print(f"📊 Produk dengan harga valid: {valid_price_products}")
    print(f"📊 Produk dengan rating valid: {valid_rating_products}")

    # Statistik basic jika ada data valid
    if valid_price_products > 0:
        price_stats = df_clean.filter(F.col("discounted_price_numeric").isNotNull()) \
                             .agg(F.min("discounted_price_numeric").alias("min_price"),
                                  F.max("discounted_price_numeric").alias("max_price"),
                                  F.avg("discounted_price_numeric").alias("avg_price")) \
                             .collect()[0]

        print(f"💰 Rentang harga: ₹{price_stats['min_price']:.0f} - ₹{price_stats['max_price']:.0f}")
        print(f"💰 Rata-rata harga: ₹{price_stats['avg_price']:.0f}")

    if valid_rating_products > 0:
        rating_stats = df_clean.filter(F.col("rating_numeric").isNotNull()) \
                              .agg(F.avg("rating_numeric").alias("avg_rating")) \
                              .collect()[0]['avg_rating']

        print(f"⭐ Rata-rata rating: {rating_stats:.2f}")

except Exception as e:
    print(f"Error dalam ringkasan: {e}")

# FUNGSI HELPER UNTUK ANALISIS LANJUTAN
def analyze_top_products(df, price_col="discounted_price_numeric", rating_col="rating_numeric", limit=5):
    """
    Fungsi untuk menganalisis produk terbaik berdasarkan harga dan rating.

    Mengapa fungsi ini berguna:
    1. Reusability: Dapat digunakan berkali-kali dengan parameter berbeda
    2. Maintainability: Mudah dimodifikasi jika logic berubah
    3. Readability: Kode lebih bersih dan mudah dipahami
    """
    try:
        # Produk dengan rating tertinggi
        top_rated = df.filter(F.col(rating_col).isNotNull()) \
                     .orderBy(F.desc(rating_col)) \
                     .select("product_name", price_col, rating_col) \
                     .limit(limit)

        print(f"\n🏆 TOP {limit} PRODUK RATING TERTINGGI:")
        top_rated.show(limit, truncate=False)

        # Produk termurah dengan rating bagus (≥4.0)
        best_value = df.filter((F.col(rating_col) >= 4.0) &
                              (F.col(price_col).isNotNull())) \
                       .orderBy(F.asc(price_col)) \
                       .select("product_name", price_col, rating_col) \
                       .limit(limit)

        print(f"\n💎 TOP {limit} PRODUK VALUE TERBAIK (Rating ≥4.0, Harga Termurah):")
        best_value.show(limit, truncate=False)

        return top_rated, best_value

    except Exception as e:
        print(f"Error dalam analisis top products: {e}")
        return None, None

# Jalankan analisis top products
if 'discounted_price_numeric' in df_clean.columns and 'rating_numeric' in df_clean.columns:
    top_rated, best_value = analyze_top_products(df_clean)

print("\n🎯 BEST PRACTICES YANG DITERAPKAN:")
print("=" * 50)
print("✅ Menggunakan F.col() untuk menghindari konflik namespace")
print("✅ Error handling pada setiap operasi critical")
print("✅ Data validation sebelum operasi numerik")
print("✅ Flexible brand detection dengan regex")
print("✅ Sample data untuk visualisasi performa yang baik")
print("✅ Modular functions untuk reusability")

# Tutup Spark session
spark.stop()
print("\n🛑 Spark session ditutup dengan sukses!")

🔧 Menggunakan import yang lebih spesifik untuk menghindari konflik nama
🚀 Spark Session berhasil dibuat!
Spark Version: 3.5.1
❌ Error saat membaca file: [PATH_NOT_FOUND] Path does not exist: file:/content/amazon - Copy.csv.
✅ Data dimuat dengan opsi sederhana

📊 INFORMASI DASAR DATASET
Jumlah Baris: 1465
Jumlah Kolom: 16

📋 KOLOM YANG TERSEDIA:
 1. product_id
 2. product_name
 3. category
 4. discounted_price
 5. actual_price
 6. discount_percentage
 7. rating
 8. rating_count
 9. about_product
10. user_id
11. user_name
12. review_id
13. review_title
14. review_content
15. img_link
16. product_link

🔍 SCHEMA DATASET
root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- discounted_price: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- discount_percentage: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_count: string (nullable = true)
 |-- about_product: 

✅ Histogram harga berhasil dibuat


✅ Histogram rating berhasil dibuat


✅ Scatter plot berhasil dibuat

💡 RINGKASAN ANALISIS
📊 Total produk: 1465
📊 Produk dengan harga valid: 1408
📊 Produk dengan rating valid: 1407
💰 Rentang harga: ₹39 - ₹77990
💰 Rata-rata harga: ₹3139
⭐ Rata-rata rating: 4.10

🏆 TOP 5 PRODUK RATING TERTINGGI:
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+--------------+
|product_name                                                                                                                                                                                                                                                                |discounted_price_numeric|rating_numeric|
+------------------------------------------------------------------------------------------------------------------------